<h1>SI 670 Kaggle Competition 4</h1>

Team GSIs

In [30]:
import os
import matplotlib.pyplot as plt
import numpy as np 

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import Model, Sequential, layers
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.callbacks import EarlyStopping

<h2>Load datasets</h2>

In [2]:
len(list(os.listdir("../data/images_train/bathroom"))), len(list(os.listdir("../data/images_train/kitchen")))

(301, 315)

In [3]:
data = tf.keras.utils.image_dataset_from_directory("../data/images_train", 
                                                   image_size = (512, 512))

Found 616 files belonging to 2 classes.


In [4]:
X, y = zip(*data)

X = np.concatenate(X)
y = np.concatenate(y)

In [5]:
len(X), len(y)

(616, 616)

In [31]:
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state = 42, test_size = 0.7)

<h2>Preprocess Images</h2>

In [9]:
X = np.apply_along_axis(lambda s: s / 255.0, axis = 1, arr = X)

<h2>Train Model</h2>

In [23]:
model = Sequential([
    tf.keras.layers.Conv2D(64, (5, 5), activation = 'relu', input_shape = (512, 512, 3)), 
    tf.keras.layers.MaxPooling2D((2, 2)), 
    tf.keras.layers.Conv2D(128, (5, 5), activation = 'relu'), 
    tf.keras.layers.MaxPooling2D((2, 2)), 
    tf.keras.layers.Conv2D(128, (5, 5), activation = 'relu'), 
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(64, activation = 'relu'), 
    tf.keras.layers.Dense(2)
])

In [28]:
loss = BinaryCrossentropy()
model.compile(optimizer = 'adam', loss = loss, metrics = ['accuracy'])

In [33]:
early_stopping = EarlyStopping(monitor = 'val_loss', min_delta = 0.01, patience = 3, restore_best_weights = True)

model_history = model.fit(X_train, y_train, epochs = 50, batch_size = 32, 
                          callbacks = [early_stopping], 
                          validation_data = (X_val, y_val))

SyntaxError: unmatched ')' (3757403320.py, line 1)